<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/Text_Preprocessing_Tools_for_Korean_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **한국어 전처리 패키지**
---

<br/>

## PyKoSpacing(띄어쓰기)
---
> 딥러닝 기반 한국어 띄어쓰기 모델

<br/>

## Py-Hanspell(맞춤법)
---
> 네이버 한글 맞춤법 검사기 모델

> PyKoSpacing과 같이 띄어쓰기도 지원

<br/>

## **SOYNLP를 이용한 단어 토큰화**
---
> 기존 형태소 분석기와 달리 신조어도 구분할 수 있다

> 비지도학습 기반의 단어 토크나이저이므로 사용 전 코퍼스 학습이 필요하다

> **응집 확률** : 함께 응집하여 나타나는 빈도가 높은 문자열일수록 단어일 확률이 높다

> **브랜칭 엔트로피** : 다음에 올 문자를 예측 가능한 정도가 단어를 판단의 지표가 된다

<br/>

## SOYNLP -> L tokenizer
---
> 한국어 띄어쓰기 단위로 나눈 어절 토큰은 주로 L 토큰 + R 토큰의 형식을 가진다

> 분리 기준으로 L 토큰을 찾아내는 원리

<br/>

## SOYNLP -> Max Score tokenizer
---
> 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 문자열을 단어로 구분해 토큰화한다

<br/>

## SOYNLP -> Normalizer
---
> ㅋㅋ, ㅎㅎ 등의 반복되는 이모티콘 또는 의미 없게 반복되는 문자열을 정규화

<br/>

## Customized KoNLPy
---
> 형태소 분석기에 없는 단어를 구분하기 위해 사용자 사전을 쉽게 추가할 수 있음



In [ ]:

pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
# PyKoSpacing(한국어 띄어쓰기 패키지)

from pykospacing import spacing

text = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_text = text.replace(' ', '')
print(new_text)
print()

kospacing_text = spacing(new_text)
print(text)
print()
print(kospacing_text)

In [ ]:
pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
# Py-Hanspell(네이버 맞춤법 검사기)

from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지"
spelled_sent = spell_checker.check(sent)
hanspell_sent = spelled_sent.checked

print(hanspell_sent)
print()

sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(' ', '')
spelled_sent = spell_checker.check(new_sent)
hanspell_sent = spelled_sent.checked

print(hanspell_sent)

In [ ]:
pip install soynlp

In [ ]:
# SOYNLP를 이용한 단어 토큰화

import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

corpus = DoublespaceLineCorpus("2016-10-20.txt")
print(len(corpus))
print()

i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    print()
    i += 1
  if i == 3:
    break

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

In [ ]:
# SOYNLP의 응집 확률

print(word_score_table["반포한"].cohesion_forward)
print(word_score_table["반포한강"].cohesion_forward)
print(word_score_table["반포한강공"].cohesion_forward)
print(word_score_table["반포한강공원"].cohesion_forward)
print(word_score_table["반포한강공원에"].cohesion_forward)
print()

print(word_score_table["디스"].right_branching_entropy) # 디스코드, 디스카운트 등 다양함
print(word_score_table["디스플"].right_branching_entropy) # 디스플레이가 명백
print(word_score_table["디스플레"].right_branching_entropy)
print(word_score_table["디스플레이"].right_branching_entropy) # 조사나 다른 단어가 뒤에 붙을 수 있어 엔트로피가 높음

In [ ]:
# SOYNLP -> L tokenizer

from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores = scores)
l_tokenized = l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten = False)

print(l_tokenized)

In [ ]:
# SOYNLP -> Max Score tokenizer

from soynlp.tokenizer import MaxScoreTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
maxscore_tokenizer = MaxScoreTokenizer(scores = scores)
maxscore_tokenized = maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

print(maxscore_tokenized)

In [ ]:
# 반복되는 문자 정제

from soynlp.normalizer import *

print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print()

print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))

In [ ]:
pip install customized_konlpy

In [ ]:
# 형태소 분석기에 사용자 사전 추가

from ckonlpy.tag import Twitter

twitter = Twitter()
print(twitter.morphs('은경이는 사무실로 갔습니다.'))
print()

twitter.add_dictionary('은경이', 'Noun')
print(twitter.morphs('은경이는 사무실로 갔습니다.'))